In [1]:
import numpy as np
import tensorflow as tf

initial = [np.random.random_sample(), np.random.random_sample()]

/Users/clint/virtualenv/tf/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## First-order Minimization

In [2]:
sess = tf.InteractiveSession()

vs = tf.Variable(initial, dtype='float32')
f = vs[0]**2 + 2.0*vs[0]*vs[1] + 3.0*vs[1]**2 + 4.0*vs[0] + 5.0*vs[1] + 6.0
sess.run(tf.global_variables_initializer())

learning_rate = 0.2
for e in range(10):
    print('EPOCH', e, 'f=%.3f'%f.eval(), 'variables=', ['%.3f'%v for v in vs.eval()])
    
    grads = tf.gradients(f, vs)[0]
    delta = - grads*learning_rate
    print('\t \tgradients=', ['%.3f'%g for g in grads.eval()],
          'delta=', ['%.3f'%d for d in delta.eval()])
    op = vs.assign( vs + delta )
    op.eval()

print('FINAL  ', 'f=%.3f'%f.eval(), 'variables=', ['%.3f'%v for v in vs.eval()])
sess.close()

EPOCH 0 f=10.224 variables= ['0.468', '0.311']
	 	gradients= ['5.557', '7.800'] delta= ['-1.111', '-1.560']
EPOCH 1 f=3.884 variables= ['-0.644', '-1.249']
	 	gradients= ['0.214', '-3.783'] delta= ['-0.043', '0.757']
EPOCH 2 f=2.667 variables= ['-0.686', '-0.493']
	 	gradients= ['1.642', '0.671'] delta= ['-0.328', '-0.134']
EPOCH 3 f=2.287 variables= ['-1.015', '-0.627']
	 	gradients= ['0.717', '-0.791'] delta= ['-0.143', '0.158']
EPOCH 4 f=2.110 variables= ['-1.158', '-0.469']
	 	gradients= ['0.746', '-0.128'] delta= ['-0.149', '0.026']
EPOCH 5 f=2.012 variables= ['-1.307', '-0.443']
	 	gradients= ['0.499', '-0.273'] delta= ['-0.100', '0.055']
EPOCH 6 f=1.955 variables= ['-1.407', '-0.388']
	 	gradients= ['0.409', '-0.145'] delta= ['-0.082', '0.029']
EPOCH 7 f=1.922 variables= ['-1.489', '-0.359']
	 	gradients= ['0.303', '-0.134'] delta= ['-0.061', '0.027']
EPOCH 8 f=1.903 variables= ['-1.550', '-0.333']
	 	gradients= ['0.236', '-0.094'] delta= ['-0.047', '0.019']
EPOCH 9 f=1.891 vari

## Second-order Minimization

In [3]:
sess = tf.InteractiveSession()

vs = tf.Variable(initial)
f = vs[0]**2 + 2.0*vs[0]*vs[1] + 3.0*vs[1]**2 + 4.0*vs[0] + 5.0*vs[1] + 6.0
sess.run(tf.global_variables_initializer())

for e in range(3):
    print('EPOCH', e, 'f=%.3f'%f.eval(), 'variables=', ['%.3f'%v for v in vs.eval()])
    
    grads = tf.gradients(f, vs)[0]
    hessi = tf.hessians(f, vs)[0]
    hessi_inv = tf.matrix_inverse(hessi)
    delta = - tf.reshape( tf.matmul(hessi_inv, tf.reshape(grads, [-1, 1])), [2] )
    print('\t \tgradients=', ['%.3f'%g for g in grads.eval()],
          'hessi_inv=', ('%s'%hessi_inv.eval()).replace('\n', ','),
          'delta=', ['%.3f'%d for d in delta.eval()])

    op = vs.assign( vs + delta )
    op.eval()

print('FINAL  ', 'f=%.3f'%f.eval(), 'variables=', ['%.3f'%v for v in vs.eval()])
sess.close()

EPOCH 0 f=10.224 variables= ['0.468', '0.311']
	 	gradients= ['5.557', '7.800'] hessi_inv= [[ 0.75 -0.25], [-0.25  0.25]] delta= ['-2.218', '-0.561']
EPOCH 1 f=1.875 variables= ['-1.750', '-0.250']
	 	gradients= ['-0.000', '-0.000'] hessi_inv= [[ 0.75 -0.25], [-0.25  0.25]] delta= ['0.000', '-0.000']
EPOCH 2 f=1.875 variables= ['-1.750', '-0.250']
	 	gradients= ['0.000', '0.000'] hessi_inv= [[ 0.75 -0.25], [-0.25  0.25]] delta= ['-0.000', '-0.000']
FINAL   f=1.875 variables= ['-1.750', '-0.250']


## Second-order Minimization self implemented Hessian

In [4]:
from functools import reduce
from tensorflow.python.ops import gradients_impl
from tensorflow.python.ops import array_ops, tensor_array_ops, control_flow_ops

def hessians_highrank(ys, xs, gradients=None, name="hessians", colocate_gradients_with_ops=False,
            gate_gradients=False, aggregation_method=None):
  """Constructs the Hessian (one or more rank matrix) of sum of `ys` with respect to `x` in `xs`.
  `hessians_highrank()` adds ops to the graph to output the Hessian matrix of `ys`
  with respect to `xs`.  It returns a list of `Tensor` of length `len(xs)`
  where each tensor is the Hessian of `sum(ys)`. This function currently
  only supports evaluating the Hessian with respect to (a list of) one-
  dimensional tensors.
  The Hessian is a matrix of second-order partial derivatives of a scalar
  tensor (see https://en.wikipedia.org/wiki/Hessian_matrix for more details).
  Args:
    ys: A `Tensor` or list of tensors to be differentiated.
    xs: A `Tensor` or list of tensors to be used for differentiation.
    name: Optional name to use for grouping all the gradient ops together.
      defaults to 'hessians'.
    colocate_gradients_with_ops: See `gradients()` documentation for details.
    gate_gradients: See `gradients()` documentation for details.
    aggregation_method: See `gradients()` documentation for details.
  Returns:
    A list of Hessian matrices of `sum(ys)` for each `x` in `xs`.
  Raises:
    LookupError: if one of the operations between `xs` and `ys` does not
      have a registered gradient function.
  """
  xs = gradients_impl._AsList(xs)
  kwargs = {
    'colocate_gradients_with_ops': colocate_gradients_with_ops,
    'gate_gradients': gate_gradients,
    'aggregation_method': aggregation_method
  }
  # Compute first-order derivatives and iterate for each x in xs.
  hessians = []
  _gradients = tf.gradients(ys, xs, **kwargs) if gradients is None else gradients
  for i, _gradient, x in zip(range(len(xs)), _gradients, xs):
    shape = x.shape
    _gradient = tf.reshape(_gradient, [-1])
    
    n = tf.size(x)
    loop_vars = [
      array_ops.constant(0, tf.int32),
      tensor_array_ops.TensorArray(x.dtype, n)
    ]
    _, hessian = control_flow_ops.while_loop(
      lambda j, _: j < n,
      lambda j, result: (j + 1, result.write(j, tf.gradients(_gradient[j], x, **kwargs)[0])),
      loop_vars
    )
    hessians.append(hessian.stack())
  return hessians

In [5]:
sess = tf.InteractiveSession()

vs = tf.Variable(initial)
f = vs[0]**2 + 2.0*vs[0]*vs[1] + 3.0*vs[1]**2 + 4.0*vs[0] + 5.0*vs[1] + 6.0
sess.run(tf.global_variables_initializer())

for e in range(3):
    print('EPOCH', e, 'f=%.3f'%f.eval(), 'variables=', ['%.3f'%v for v in vs.eval()])
    
    grads = tf.gradients(f, vs)[0]
    hessi = hessians_highrank(f, vs)[0]
    hessi_inv = tf.matrix_inverse(hessi)
    delta = - tf.reshape( tf.matmul(hessi_inv, tf.reshape(grads, [-1, 1])), [2] )
    print('\t \tgradients=', ['%.3f'%g for g in grads.eval()],
          'hessi_inv=', ('%s'%hessi_inv.eval()).replace('\n', ','),
          'delta=', ['%.3f'%d for d in delta.eval()])

    op = vs.assign( vs + delta )
    op.eval()

print('FINAL  ', 'f=%.3f'%f.eval(), 'variables=', ['%.3f'%v for v in vs.eval()])
sess.close()

EPOCH 0 f=10.224 variables= ['0.468', '0.311']
	 	gradients= ['5.557', '7.800'] hessi_inv= [[ 0.75 -0.25], [-0.25  0.25]] delta= ['-2.218', '-0.561']
EPOCH 1 f=1.875 variables= ['-1.750', '-0.250']
	 	gradients= ['-0.000', '-0.000'] hessi_inv= [[ 0.75 -0.25], [-0.25  0.25]] delta= ['0.000', '-0.000']
EPOCH 2 f=1.875 variables= ['-1.750', '-0.250']
	 	gradients= ['0.000', '0.000'] hessi_inv= [[ 0.75 -0.25], [-0.25  0.25]] delta= ['-0.000', '-0.000']
FINAL   f=1.875 variables= ['-1.750', '-0.250']


## Second-order Minimization self implemented Diagonal Hessian

### brute force

In [6]:
sess = tf.InteractiveSession()

vs = tf.Variable(initial)
f = vs[0]**2 + 2.0*vs[0]*vs[1] + 3.0*vs[1]**2 + 4.0*vs[0] + 5.0*vs[1] + 6.0
sess.run(tf.global_variables_initializer())

for e in range(10):
    print('EPOCH', e, 'f=%.3f'%f.eval(), 'variables=', ['%.3f'%v for v in vs.eval()])
    
    grads = tf.gradients(f, vs, stop_gradients=vs)[0]
    hessi = hessians_highrank(f, vs)[0] * tf.eye(2)
    
    hessi_inv = tf.matrix_inverse(hessi)
    delta = - tf.reshape( tf.matmul(hessi_inv, tf.reshape(grads, [-1, 1])), [2] )
    print('\t \tgradients=', ['%.3f'%g for g in grads.eval()],
          'hessi_inv=', ('%s'%hessi_inv.eval()).replace('   ', '').replace('\n', ','),
          'delta=', ['%.3f'%d for d in delta.eval()])

    op = vs.assign( vs + delta )
    op.eval()

print('FINAL  ', 'f=%.3f'%f.eval(), 'variables=', ['%.3f'%v for v in vs.eval()])
sess.close()

EPOCH 0 f=10.224 variables= ['0.468', '0.311']
	 	gradients= ['5.557', '7.800'] hessi_inv= [[ 0.50.  ], [ 0. 0.16666667]] delta= ['-2.779', '-1.300']
EPOCH 1 f=4.658 variables= ['-2.311', '-0.989']
	 	gradients= ['-2.600', '-5.557'] hessi_inv= [[ 0.50.  ], [ 0. 0.16666667]] delta= ['1.300', '0.926']
EPOCH 2 f=2.803 variables= ['-1.011', '-0.063']
	 	gradients= ['1.852', '2.600'] hessi_inv= [[ 0.50.  ], [ 0. 0.16666667]] delta= ['-0.926', '-0.433']
EPOCH 3 f=2.184 variables= ['-1.937', '-0.496']
	 	gradients= ['-0.867', '-1.852'] hessi_inv= [[ 0.50.  ], [ 0. 0.16666667]] delta= ['0.433', '0.309']
EPOCH 4 f=1.978 variables= ['-1.504', '-0.188']
	 	gradients= ['0.617', '0.867'] hessi_inv= [[ 0.50.  ], [ 0. 0.16666667]] delta= ['-0.309', '-0.144']
EPOCH 5 f=1.909 variables= ['-1.812', '-0.332']
	 	gradients= ['-0.289', '-0.617'] hessi_inv= [[ 0.50.  ], [ 0. 0.16666667]] delta= ['0.144', '0.103']
EPOCH 6 f=1.886 variables= ['-1.668', '-0.229']
	 	gradients= ['0.206', '0.289'] hessi_inv= [[ 

### optimized computational impl.

In [7]:
from functools import reduce
from tensorflow.python.ops import gradients_impl
from tensorflow.python.ops import array_ops, tensor_array_ops, control_flow_ops

def diagonal_inverse_hessians_highrank(ys, xs, gradients=None, name="hessians", colocate_gradients_with_ops=False,
            gate_gradients=False, aggregation_method=None):
  """Constructs the Hessian (one or more rank matrix) of sum of `ys` with respect to `x` in `xs`.
  `hessians_highrank()` adds ops to the graph to output the Hessian matrix of `ys`
  with respect to `xs`.  It returns a list of `Tensor` of length `len(xs)`
  where each tensor is the Hessian of `sum(ys)`. This function currently
  only supports evaluating the Hessian with respect to (a list of) one-
  dimensional tensors.
  The Hessian is a matrix of second-order partial derivatives of a scalar
  tensor (see https://en.wikipedia.org/wiki/Hessian_matrix for more details).
  Args:
    ys: A `Tensor` or list of tensors to be differentiated.
    xs: A `Tensor` or list of tensors to be used for differentiation.
    name: Optional name to use for grouping all the gradient ops together.
      defaults to 'hessians'.
    colocate_gradients_with_ops: See `gradients()` documentation for details.
    gate_gradients: See `gradients()` documentation for details.
    aggregation_method: See `gradients()` documentation for details.
  Returns:
    A list of Hessian matrices of `sum(ys)` for each `x` in `xs`.
  Raises:
    LookupError: if one of the operations between `xs` and `ys` does not
      have a registered gradient function.
  """
  xs = gradients_impl._AsList(xs)
  kwargs = {
    'colocate_gradients_with_ops': colocate_gradients_with_ops,
    'gate_gradients': gate_gradients,
    'aggregation_method': aggregation_method
  }
  # Compute first-order derivatives and iterate for each x in xs.
  hessians = []
  _gradients = tf.gradients(ys, xs, **kwargs) if gradients is None else gradients
  for i, _gradient, x in zip(range(len(xs)), _gradients, xs):
    shape = x.shape
    _gradient = tf.reshape(_gradient, [-1])
    
    n = tf.size(x)
    g = tf.gradients(_gradient*tf.eye(n), x)[0]
    hessian = tf.diag( 1.0 / tf.reshape(g, [-1]) )
    hessians.append(hessian)
  return hessians

In [8]:
sess = tf.InteractiveSession()

vs = tf.Variable(initial)
f = vs[0]**2 + 2.0*vs[0]*vs[1] + 3.0*vs[1]**2 + 4.0*vs[0] + 5.0*vs[1] + 6.0
sess.run(tf.global_variables_initializer())

for e in range(10):
    print('EPOCH', e, 'f=%.3f'%f.eval(), 'variables=', ['%.3f'%v for v in vs.eval()])
    
    grads = tf.gradients(f, vs, stop_gradients=vs)[0]
    hessi_inv = diagonal_inverse_hessians_highrank(f, vs)[0]

    delta = - tf.reshape( tf.matmul(hessi_inv, tf.reshape(grads, [-1, 1])), [2] )
    print('\t \tgradients=', ['%.3f'%g for g in grads.eval()],
          'hessi_inv=', ('%s'%hessi_inv.eval()).replace('   ', '').replace('\n', ','),
          'delta=', ['%.3f'%d for d in delta.eval()])

    op = vs.assign( vs + delta )
    op.eval()

print('FINAL  ', 'f=%.3f'%f.eval(), 'variables=', ['%.3f'%v for v in vs.eval()])
sess.close()

EPOCH 0 f=10.224 variables= ['0.468', '0.311']
	 	gradients= ['5.557', '7.800'] hessi_inv= [[ 0.250.], [ 0.  0.125]] delta= ['-1.389', '-0.975']
EPOCH 1 f=2.390 variables= ['-0.921', '-0.664']
	 	gradients= ['0.829', '-0.829'] hessi_inv= [[ 0.250.], [ 0.  0.125]] delta= ['-0.207', '0.104']
EPOCH 2 f=2.165 variables= ['-1.129', '-0.561']
	 	gradients= ['0.621', '-0.621'] hessi_inv= [[ 0.250.], [ 0.  0.125]] delta= ['-0.155', '0.078']
EPOCH 3 f=2.038 variables= ['-1.284', '-0.483']
	 	gradients= ['0.466', '-0.466'] hessi_inv= [[ 0.250.], [ 0.  0.125]] delta= ['-0.117', '0.058']
EPOCH 4 f=1.967 variables= ['-1.400', '-0.425']
	 	gradients= ['0.350', '-0.350'] hessi_inv= [[ 0.250.], [ 0.  0.125]] delta= ['-0.087', '0.044']
EPOCH 5 f=1.927 variables= ['-1.488', '-0.381']
	 	gradients= ['0.262', '-0.262'] hessi_inv= [[ 0.250.], [ 0.  0.125]] delta= ['-0.066', '0.033']
EPOCH 6 f=1.904 variables= ['-1.553', '-0.348']
	 	gradients= ['0.197', '-0.197'] hessi_inv= [[ 0.250.], [ 0.  0.125]] delta=

In [9]:
sess = tf.InteractiveSession()

vs = tf.Variable(initial)
f = vs[0]**2 + 2.0*vs[0]*vs[1] + 3.0*vs[1]**2 + 4.0*vs[0] + 5.0*vs[1] + 6.0
sess.run(tf.global_variables_initializer())

for e in range(10):
    print('EPOCH', e, 'f=%.3f'%f.eval(), 'variables=', ['%.3f'%v for v in vs.eval()])
    
    grads = tf.gradients(f, vs, stop_gradients=vs)[0]
    hessi_inv = diagonal_inverse_hessians_highrank(f, vs)[0]

    delta = - tf.reshape( tf.matmul(hessi_inv, tf.reshape(grads, [-1, 1])), [2] )
    print('\t \tgradients=', ['%.3f'%g for g in grads.eval()],
          'hessi_inv=', ('%s'%hessi_inv.eval()).replace('   ', '').replace('\n', ','),
          'delta=', ['%.3f'%d for d in delta.eval()])

    op = vs.assign( vs + (0.5*delta) )
    op.eval()

print('FINAL  ', 'f=%.3f'%f.eval(), 'variables=', ['%.3f'%v for v in vs.eval()])
sess.close()

EPOCH 0 f=10.224 variables= ['0.468', '0.311']
	 	gradients= ['5.557', '7.800'] hessi_inv= [[ 0.250.], [ 0.  0.125]] delta= ['-1.389', '-0.975']
EPOCH 1 f=4.434 variables= ['-0.227', '-0.177']
	 	gradients= ['3.193', '3.486'] hessi_inv= [[ 0.250.], [ 0.  0.125]] delta= ['-0.798', '-0.436']
EPOCH 2 f=2.876 variables= ['-0.626', '-0.395']
	 	gradients= ['1.959', '1.380'] hessi_inv= [[ 0.250.], [ 0.  0.125]] delta= ['-0.490', '-0.173']
EPOCH 3 f=2.402 variables= ['-0.871', '-0.481']
	 	gradients= ['1.297', '0.373'] hessi_inv= [[ 0.250.], [ 0.  0.125]] delta= ['-0.324', '-0.047']
EPOCH 4 f=2.219 variables= ['-1.033', '-0.504']
	 	gradients= ['0.926', '-0.091'] hessi_inv= [[ 0.250.], [ 0.  0.125]] delta= ['-0.231', '0.011']
EPOCH 5 f=2.123 variables= ['-1.149', '-0.499']
	 	gradients= ['0.706', '-0.288'] hessi_inv= [[ 0.250.], [ 0.  0.125]] delta= ['-0.176', '0.036']
EPOCH 6 f=2.061 variables= ['-1.237', '-0.481']
	 	gradients= ['0.565', '-0.357'] hessi_inv= [[ 0.250.], [ 0.  0.125]] delta=